In [5]:
import multiprocessing as mp
import pandas as pd
import sys
import os
sys.path.append("../code/")

from model import TemporalHypergraphModel
from model import run_from_df_and_save_edgelists, read_edgelists_from_df

import matplotlib.pyplot as plt
%matplotlib inline

### Preparing the parameters space
I create a dataframe with all the parameters of the simulations I want to perform

In [6]:
#Directory for results
OUT_DIR = 'results/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

parnames = ["N","t_max","beta","alpha","n0","L","epsilon"]

#Fixed model parameters
N = 700
t_max = 2000
beta = 0.8
L = 1

#Variable model parameters
alphas = [0.05, 0.2, 0.4, 0.6, 0.8, 0.95]
n0s = [3,5,7,8,9,10,11,12,13,14]
epsilons = [1,5,10,15,20,25,30]

#Parameters index
n_combinations = len(alphas)*len(n0s)*len(epsilons)
parsindexes = range(n_combinations)

In [7]:
pars_df = pd.DataFrame(columns=parnames, index=parsindexes)
pars_df.index.name = 'pars_id'

parsindex = 0
for alpha in alphas:
    for n0 in n0s:
        for epsilon in epsilons:
            
            pars_df.loc[parsindex] = pd.Series({"N":N,
                                                "t_max":t_max,
                                                "beta":beta,
                                                "alpha":alpha,
                                                "n0":n0,
                                                "L":L,
                                                "epsilon":epsilon})
            parsindex+=1
            
pars_df['N'] = pars_df['N'].astype(int)
pars_df['t_max'] = pars_df['t_max'].astype(int)
pars_df['epsilon'] = pars_df['epsilon'].astype(int)
pars_df.head()

,N,t_max,beta,alpha,n0,L,epsilon
pars_id,,,,,,,
0,700,2000,0.8,0.05,3.0,1.0,1
1,700,2000,0.8,0.05,3.0,1.0,5
2,700,2000,0.8,0.05,3.0,1.0,10
3,700,2000,0.8,0.05,3.0,1.0,15
4,700,2000,0.8,0.05,3.0,1.0,20


Saving the dataframe with the parameters

In [8]:
pars_df.to_csv(OUT_DIR+'parameters.csv')

## Running the model

In [59]:
#Reading the parameter dataframe
PATH = 'results/'

pars_df = pd.read_csv(PATH+'parameters.csv')
pars_df.set_index('pars_id', inplace=True)
pars_df.head()

,N,t_max,beta,alpha,n0,L,epsilon
pars_id,,,,,,,
0,700,2000,0.8,0.05,3.0,1.0,1
1,700,2000,0.8,0.05,3.0,1.0,5
2,700,2000,0.8,0.05,3.0,1.0,10
3,700,2000,0.8,0.05,3.0,1.0,15
4,700,2000,0.8,0.05,3.0,1.0,20


In [ ]:
#Simulation parameters
n_processes = mp.cpu_count()

pool = mp.Pool(n_processes)

for pars_id in pars_df.index: 
    pool.apply_async(run_from_df_and_save_edgelists, args=(pars_id, pars_df, PATH))
pool.close()